# Cutomizing resnet50_mask_rcnn from detectron2 to deepfashion2 dataset

## **Importing dependencies**

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

## **registering datasets**

In [ ]:
#registering train data
register_coco_instances("fashion_train", {}, 
                        "/path/to/json/file/train.json",
                        "/path/to/dataset/train/image/")

fashion_metadata = MetadataCatalog.get("fashion_train")
print(fashion_metadata)
dataset_dicts = DatasetCatalog.get("fashion_train")
print(dataset_dicts)

#registering test data
register_coco_instances("fashion_test", {}, 
                        "/path/to/json/file/test.json",
                        "/path/to/testset/image/")


fashion_test_metadata = MetadataCatalog.get("fashion_test")
print(fashion_test_metadata)
dataset_test_dicts = DatasetCatalog.get("fashion_test")
print(dataset_test_dicts)

## **visualizing ground truths**

In [ ]:
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    v = Visualizer(im[:, :, ::-1], metadata=fashion_metadata, scale=0.8)
    v = v.draw_dataset_dict(d)
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

## **setting up model's configuration**

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.OUTPUT_DIR = "/path/to/log/folder/"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("fashion_train",)
cfg.DATASETS.TEST = ("fashion_test")
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001  # pick a good LR
cfg.SOLVER.MAX_ITER = 271740 #13 587*20epc
cfg.SOLVER.STEPS = (163044,230979) # The iteration number to decrease learning rate by GAMMA. to decay lr (12,17)
cfg.SOLVER.gamma = 0.5
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.CHECKPOINT_PERIOD = 13587 #after each epc        # do not decay learning rate

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5 

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## **launch training**

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## **evaluation**

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)
#%%
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("fashion_test",output_dir="/path/to/results/folder/")
val_loader = build_detection_test_loader(cfg, "fashion_test")
data = inference_on_dataset(predictor.model, val_loader, evaluator)
print(data)

## **Inference**

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode , _create_text_labels 
from detectron2.utils.colormap import random_color
import matplotlib.pyplot as plt
import cv2.cv2 as cv2


img_path="/image/path/exemple.jpg"

cfg = get_cfg()
cfg.MODEL.WEIGHTS = "/path/to/final/check^point/model_final.pth" # path for final model
predictor = DefaultPredictor(cfg)
im = cv2.imread(img_path)
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
               metadata=fashion_test_metadata,
               scale=1,
               instance_mode=ColorMode.SEGMENTATION)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#.draw_text(outputs["instances"].pred_classes.to("cpu"),(x,y),font_size=12)
#out=v.draw_text(outputs["instances"].pred_classes.to("cpu"),(x,y),font_size=20)
img = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
plt.figure(figsize = (15, 12))
plt.axis('off')
plt.imshow(img)